In [1]:
import sys
sys.path.append("..")

from scripts.utils import DuckdbUtils

duckdb_utils = DuckdbUtils()

In [2]:
sql_query = """
SHOW tables
"""

duckdb_utils.run_sql_query_and_return_df(sql_query)

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,first_model
3,gold_2023_agency_fee_metrics
4,gold_2023_county_violations_day_of_week_heatmap
5,gold_2023_precinct_violations_day_of_week_heatmap
6,gold_2023_ticket_counts_year_month
7,gold_2023_ticket_counts_year_week
8,gold_2023_violations_day_of_week_heatmap
9,gold_2023_weekly_violations_day_of_week_heatmap
